<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/0-outer-joins/1-outer-join-sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reviewing SQL Joins - Inner and Outer Joins

When performing joins, we have multiple types, an inner join, an outer join, and a full outer join.  In this lesson, we'll review the different kinds of joins and learn use cases for them.

### Upgrading Sqlite

Because we would like to explore full outer joins in this article, we will need to upgrade our version of sqlite3 (or load our data to a local database like postgres).

> **Warning**: The following code will only work on colab, and probably will not work locally.   

In [2]:
from time import process_time
start_time = process_time()
import subprocess
try:
    import google.colab # if colab exists, install pysqlite-binary
    subprocess.run(['pip', 'install', 'pysqlite3-binary'], capture_output=False)
    import pysqlite3 as sqlite3

except ModuleNotFoundError:
    pass
end_time = process_time()

### Loading our data

We can begin by loading our data.

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd


conn = sqlite3.connect('crm.db')
address_url = "https://raw.githubusercontent.com/tech-interviews-jigsaw/sql-advanced-joins/main/0-outer-joins/addresses.csv"
persons_url = "https://raw.githubusercontent.com/tech-interviews-jigsaw/sql-advanced-joins/main/0-outer-joins/persons.csv"
person_df = pd.read_csv(persons_url)
address_df = pd.read_csv(address_url)

In [4]:
person_df.to_sql('persons', conn, if_exists = 'replace', index = False)
address_df.to_sql('addresses', conn, if_exists = 'replace', index = False)

3

### Viewing our Data

Now let's take a look at our data.

In [4]:
pd.read_sql('select * from persons', conn)

,personId,firstName,lastName
0,1,Allen,Wang
1,2,Bob,Sanders


In [5]:
pd.read_sql('select * from addresses', conn)

,addressId,personId,city,state
0,1,2,New York City,New York
1,2,3,Leetcode,California
2,3,4,Chicago,IL


So we can see that the column joining our two tables is the personId column.

Also notice that we have some mismatches.  
* Allen Wang does not have an address
* The last two addresses of Leetcode and Chicago do not have a related person.

### Some queries

Now write a query that will only display rows where both the person *and* the related address exists.  

In [5]:
pd.read_sql('''
select * from persons p
join addresses a on p.personid = a.personid
''', conn)


# personId	firstName	lastName	addressId	personId	city	state
# 0	2	Bob	Sanders	1	2	New York City	New York
#

,personId,firstName,lastName,addressId,personId,city,state
0,2,Bob,Sanders,1,2,New York City,New York


> So as we can see, there is only one record that exists in both tables.

Next write a query that only returns firstName, lastName, city and state.  If the address information is not available for that person, return null for city and state.

In [6]:
pd.read_sql('''

select p.firstname, p.lastname,a.city, a.state
from persons p
left join addresses a on p.personid = a.personid

''', conn)

# firstName	lastName	city	state
# 0	Allen	Wang	None	None
# 1	Bob	Sanders	New York City	New York

,firstName,lastName,city,state
0,Allen,Wang,None,None
1,Bob,Sanders,New York City,New York


> Ok, so this is our left join, where we are returning all of the records in the left table even there is no corresponding match in the right table.

Using a left outer join can be useful for counting up the number of records that have missing data.  This is because `count` only counts non-null values.  

So now lets produce aggregates counting `total_persons` and `persons_with_addresses`, and `persons_without_addresses`.

> **Do not** use a case when to perform this.

In [8]:
query = '''

select count(p.personid) as total_persons,
  count(a.addressID) as persons_w_address, count(p.personid) - count(a.addressID) as persons_wo_add
from persons p
left outer join addresses a on p.personid = a.personid

'''

pd.read_sql(query, conn)

# 	total_persons	persons_w_address	persons_without
# 0	2	1	1

,total_persons,persons_w_address,persons_wo_add
0,2,1,1


And now, use a case when statement to perform this same calculation.

In [9]:
query = '''

select sum(case when p.personid then 1 else 0 end) as total_persons,
  SUM(case when a.addressID then 1 else 0 end) as persons_w_address,
  SUM(case when a.addressID is null then 1 else 0 end) as persons_wo_add
from persons p
left outer join addresses a on p.personid = a.personid


'''

pd.read_sql(query, conn)

# 	firstName	lastName	city	state	num_of_city	num_of_state
# 0	Allen	Wang	None	None	1	1

,total_persons,persons_w_address,persons_wo_add
0,2,1,1


* Full outer joins

So we saw that a a left outer join will return all of the rows from the left table.  Similarly a right outer join will return all of the rows from a right table.

> For example, the query below will return all of the address rows -- even when there is not a corresponding person row.

In [12]:
pd.read_sql('''

select *
from persons p
right join addresses a on p.personid = a.personid


''', conn)

,personId,firstName,lastName,addressId,personId,city,state
0,2.0,Bob,Sanders,1,2,New York City,New York
1,NaN,None,None,2,3,Leetcode,California
2,NaN,None,None,3,4,Chicago,IL


Ok, and if we want to return the combined set of rows from both tables, we can use a full outer join.

In [13]:
pd.read_sql("""

select *
from persons p
full outer join addresses a on p.personid = a.personid


""", conn)

,personId,firstName,lastName,addressId,personId,city,state
0,1.0,Allen,Wang,NaN,NaN,None,None
1,2.0,Bob,Sanders,1.0,2.0,New York City,New York
2,NaN,None,None,2.0,3.0,Leetcode,California
3,NaN,None,None,3.0,4.0,Chicago,IL


In [ ]:
# 	personId	firstName	lastName	addressId	personId	city	state
# 0	1.0	Allen	Wang	NaN	NaN	None	None
# 1	2.0	Bob	Sanders	1.0	2.0	New York City	New York
# 2	NaN	None	None	2.0	3.0	Leetcode	California
# 3	NaN	None	None	3.0	4.0	Chicago	IL

Notice that with the full outer join, we still specify that we want to join on the personId column.  But this time the complete set of records from both tables are returned. This because the row is displayed even when there is not a matching value.

### Summary

In this lesson, we reviewed the difference between inner joins and left outer joins.  With an inner join, a record is only returned if the primary key and foreign keys are the same.  

With a left outer join, the records on the left table are *always* returned.  And when there is no matching id on the right table, null values are returned.  

In [ ]:
pd.read_sql('''select firstName, lastName, city, state from persons left join addresses
            on persons.personId = addresses.personId''', conn)

,firstName,lastName,city,state
0,Allen,Wang,None,None
1,Bob,Alice,New York City,New York


Finally, we saw how we can count the number of present values with a simple `count` statement.  

In [12]:
query = '''select firstName, lastName, city, state, count(city) num_of_city, count(state) num_of_state
from persons left join addresses
on persons.personId = addresses.personId'''

pd.read_sql(query, conn)

,firstName,lastName,city,state,num_of_city,num_of_state
0,Allen,Wang,None,None,1,1


And we saw how we can also perform the same calculation by using a case when statement.

In [13]:
query = '''select
sum(case when persons.personid then 1 else 0 end) as total_persons,
sum(case when addressid then 1 else 0 end) as persons_with_address,
sum(case when addressid is null then 1 else 0 end) as persons_without
from persons left join addresses
on persons.personId = addresses.personId
'''

pd.read_sql(query, conn)


,total_persons,persons_with_address,persons_without
0,2,1,1
